In [1]:
import numpy as np
import copy
import itertools
import random
from statistics import mean

# Enum MTLayout

In [2]:
class Layout():
    # Main Structure:
    ## Store the current state, the number of cuts applied, and the lowest available cut position
    ## Task metrics are calculated in metric_inference function from known results
    
    def __init__(self, T, B, S):
        super().__init__()
        self.T = T
        self.B = B
        self.state = S
        if not self.check_valid():
            print("T, B, and S are not compatiable!")
            return
        
        self.num_cut = 0
        self.lowest_avail_cut = 0
        
        # Should be updated in the metric_inference function
        self.metric_list = [0.] * self.T
        self.score = 0.
        
    def check_valid(self):
        valid = True
        if len(self.state) != self.B:
            valid = False
            
        tasks = set()
        for task_set in self.state[0]:
            tasks.update(task_set)
        if len(tasks) != self.T:
            valid = False
        return valid
    
    def set_score(self):
        self.score = mean(self.metric_list)
    
    def __str__(self):
        return str(self.state)
    def __repr__(self):
        return str(self.state)
    # For sort
    def __lt__(self, other):
         return self.score < other.score

In [3]:
def enum_layout(L, layout_list):
    # Main Function:
    ##  Recursively enumerate all possible layouts
    
    # Exit Case: If the number of cut applied to L has already reached T-1, no more cut can be applied
    if L.num_cut >= L.T-1:
        return
    
    for i in range(L.lowest_avail_cut, L.B): # Cut for every avaiable block
        for task_set in L.state[i]: # Cut for each task set in ith block
            if len(task_set) == 1: # If the task set has only 1 element, it cannot be cut anymore
                continue
            subsets_list = divide_set(task_set) # Find all possible 2 subsets
            for subsets in subsets_list: # For any 2 subsets
                L_prime = apply_cut(L, i, task_set, subsets) # Construct new layout based on the cut
                print('Add a new layout!')
                print(L_prime)
                layout_list.append(L_prime)
                enum_layout(L_prime, layout_list)
    return 

In [4]:
# Helper functions
def init_S(T, B):
    # Function:
    ##  Construct the first state for given task and branch number
    ##  S is a list of list of task sets, e.g., [[{1,2,3}], [{1,2,3}], [{1,2},{3}]]
    S = []
    for i in range(B):
        task_sets = []
        task_sets.append(set([x for x in range(T)]))
        S.append(task_sets)
    return S

def divide_set(task_set):
    # Function:
    ##   Construct all possible 2 subsets for given task_set based on the binary theory
    ##   Remove duplicates and empty subset
    ##  If len(task_set) = n, len(subsets_list) = 2^(n-1) - 1
    subsets_list = []
    for pattern in itertools.product([True,False],repeat=len(task_set)):
        if pattern[0] is False and sum(pattern) != 0: # Exclude the duplicates and empty subset
            l1 = set([x[1] for x in zip(pattern,task_set) if x[0]])
            l2 = task_set - l1
            subsets = [l1, l2]
            subsets_list.append(subsets)    
    return subsets_list

def apply_cut(L, i, task_set, subsets):
    # Function:
    ## Apply a given cut on a layout to generate a new one
    L_new = copy.deepcopy(L)
    for j in range(i, L.B):
        L_new.state[j].remove(task_set) # Remove the cutted task set
        L_new.state[j] += subsets # Add the 2 new subsets of task set
    L_new.num_cut += 1 # Update the number of cuts
    L_new.lowest_avail_cut = i # Update the lowest available cut position
    return L_new

In [5]:
T = 3 # [0,1,2] tasks
B = 17 # [0,1,2,3] blocks

layout_list = [] 
S0 = init_S(T, B) # initial state
L = Layout(T, B, S0) # initial layout
layout_list.append(L)

enum_layout(L, layout_list)

Add a new layout!
[[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]
Add a new layout!
[[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
Add a new layout!
[[{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
Add a new layout!
[[{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {

Add a new layout!
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
Add a new layout!
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
Add a new layout!
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
Add a new layout!
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2},

In [7]:
len(layout_list)

511

In [10]:
layout_list[100]

[[{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]

# Derive metrics for MTState

In [26]:
# random known results
tasks = [i for i in range(T)]
two_task_metrics = {}
for two_set in itertools.combinations(tasks, 2):
    two_task_metrics[two_set] = []
    for b in range(0, B+1): # branch at B (0,...,B-1 branch point) means all share
        metric1 = random.uniform(0, 1)
        metric2 = random.uniform(0, 1)
        if b == 0:
            metric1 = metric2 = 0
        two_task_metrics[two_set].append([metric1, metric2])

In [29]:
def get_metric(two_task_metrics, subtree):
    metric_list = []
    for two_task_branch in subtree:
        branch = two_task_branch[2]
        
        two_set = (two_task_branch[0], two_task_branch[1])
        if two_set in two_task_metrics:
            metric_list.append(two_task_metrics[two_set][branch][0])

        two_set = (two_task_branch[1], two_task_branch[0])
        if two_set in two_task_metrics:
            metric_list.append(two_task_metrics[two_set][branch][1])
    return mean(metric_list)

In [30]:
def metric_inference(L, two_task_metrics):
    S = L.state
    for t1 in range(L.T):
        subtree = []
        for t2 in range(L.T):
            if t1 == t2:
                continue
            
            branch = B # No branch - All share
            for i in range(L.B): # For eac block
                share_flag = False
                for task_set in S[i]: # For each task set in ith block
                    # There exists a task set has both t1 and t2 -> t1 and t2 share in ith block
                    if t1 in task_set and t2 in task_set: 
                        share_flag = True
                        break
                if share_flag is False:
                    branch = i
                    break
            subtree.append([t1, t2, branch])
        print(subtree)
        L.metric_list[t1] = get_metric(two_task_metrics, subtree)

In [27]:
two_task_metrics

{(0, 1): [[0, 0],
  [0.3712362212414607, 0.11329006813608666],
  [0.1261700145294501, 0.5585915411086257],
  [0.8676208487328824, 0.9051551858238367],
  [0.7773142047080118, 0.8952137994676217],
  [0.6321555915995637, 0.18132797434061232],
  [0.621928730918012, 0.9506381588164967],
  [0.24884361052243442, 0.3718495989546111],
  [0.8271190602296771, 0.6117632463557817],
  [0.22060832615489345, 0.7543324250627813],
  [0.4917058238919991, 0.2263760518600867],
  [0.623225054011484, 0.028430444781461173],
  [0.8252047892637935, 0.5974842852714286],
  [0.008817045918201982, 0.47843895834310546],
  [0.705994169017754, 0.3982121093083294],
  [0.23126554774809704, 0.17417154119665812],
  [0.8565039780974186, 0.02367391855445522],
  [0.7104959528621386, 0.8926835821683775]],
 (0, 2): [[0, 0],
  [0.018770473217011974, 0.39208155506626163],
  [0.24771236176223665, 0.22209688789208615],
  [0.6701523951457212, 0.05629911892752226],
  [0.9956438195348275, 0.871207858583644],
  [0.20652856970296107, 0

In [32]:
# Run for all L
for L in layout_list:
    print(L)
    metric_inference(L, two_task_metrics)
    print(L.metric_list)

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]
[[0, 1, 17], [0, 2, 17]]
[[1, 0, 17], [1, 2, 17]]
[[2, 0, 17], [2, 1, 17]]
[0.40861678075075764, 0.7515943968963792, 0.5447267449590202]
[[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]
[[0, 1, 0], [0, 2, 0]]
[[1, 0, 0], [1, 2, 17]]
[[2, 0, 0], [2, 1, 17]]
[0, 0.30525260581219044, 0.18335737142155428]
[[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {

[[1, 0, 7], [1, 2, 13]]
[[2, 0, 7], [2, 1, 13]]
[0.3413833181599771, 0.5982450441467531, 0.31733300484844135]
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
[[0, 1, 7], [0, 2, 7]]
[[1, 0, 7], [1, 2, 14]]
[[2, 0, 7], [2, 1, 14]]
[0.3413833181599771, 0.44251931697563596, 0.1972597549158847]
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
[[0, 1, 7], [0, 2, 7]]
[[1, 0, 7], [1, 2, 15]]
[[2, 0, 7], [2, 1, 15]]
[0.3413833181599771, 0.5447933838610426, 0.23876658127960276]
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2},

In [9]:
two_task_metrics

array([[[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.34301248, 0.47809647, 0.39935949, 0.28687493],
        [0.        , 0.669081  , 0.74566342, 0.867773  , 0.16836151]],

       [[0.        , 0.34301248, 0.47809647, 0.39935949, 0.28687493],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.47743717, 0.60293637, 0.42836201, 0.78758574]],

       [[0.        , 0.669081  , 0.74566342, 0.867773  , 0.16836151],
        [0.        , 0.47743717, 0.60293637, 0.42836201, 0.78758574],
        [0.        , 0.        , 0.        , 0.        , 0.        ]]])